# Genetic Evolution Backtesting

## Introduction

The code below simulates a backtesting model using a version genetic evolution.  It starts with a panel of signals generated using vectorbt, and creates a weighted average of them across multiple individual portfolios. Then the simulation randomly adjusts the weights, and accepts the new portfolio if it is better than it's parent. There is a generational epoch setting, where at the step, all portfolios are evaluated and the top portfolio is cloned across the family. Then the simulation resumes.

In [15]:
from Backtest.models.EvolutionaryModel import EvolutionaryPortfolio, EvolutionaryPortfolioFamily, generate_weights, blend_signals
import vectorbt as vbt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from Backtest.models.VBTYFData import VBTYFData


from pathlib import Path

current_dir = Path.cwd()

np.random.seed(1337)
data_file = current_dir / "data"

## Data

Below the code just takes 1 backtest of BTC. One could scale this to take an average Sharpe Ratio across multiple time-periods in the evolution.

In [2]:
# BTC
btc_store = VBTYFData(data_file / "btc_price.csv")
btc_price = btc_store.load("BTC-USD", start="2019-01", end="2024-07")

In [3]:
btc_price

Date
2019-01-20 00:00:00+00:00     3601.013672
2019-01-21 00:00:00+00:00     3576.032471
2019-01-22 00:00:00+00:00     3604.577148
2019-01-23 00:00:00+00:00     3585.123047
2019-01-24 00:00:00+00:00     3600.865479
                                 ...     
2024-07-16 00:00:00+00:00    65097.148438
2024-07-17 00:00:00+00:00    64118.792969
2024-07-18 00:00:00+00:00    63974.066406
2024-07-19 00:00:00+00:00    66710.156250
2024-07-20 00:00:00+00:00    67163.648438
Name: Close, Length: 2009, dtype: float64

## Signals Panel

Below generates 4 signals and integrates them into a weighted average, with starting weights all equal. The code can accept a set of previously defined weights to possibly start on an old iteration. As long as the signals are arrays of boolean values, it can be added into the EvolutionaryPortfolioModel. Ideally one could add more signals and generate a larger signals panel.

In [4]:
ma_1_fast = vbt.MA.run(btc_price, 25, short_name='fast_25')
ma_1_slow = vbt.MA.run(btc_price, 100, short_name='slow_100')
ma_1_entries = ma_1_fast.ma_crossed_above(ma_1_slow)
ma_1_exits = ma_1_fast.ma_crossed_below(ma_1_slow)

ma_2_fast = vbt.MA.run(btc_price, 25, short_name='fast_30')
ma_2_slow = vbt.MA.run(btc_price, 70, short_name='slow_70')
ma_2_entries = ma_2_fast.ma_crossed_above(ma_2_slow)
ma_2_exits = ma_2_fast.ma_crossed_below(ma_2_slow)

rsi_14 = vbt.RSI.run(btc_price, window=14)
rsi_14_entries = rsi_14.rsi_crossed_below(30)
rsi_14_exits = rsi_14.rsi_crossed_above(70)

rsi_30 = vbt.RSI.run(btc_price, window=30)
rsi_30_entries = rsi_30.rsi_crossed_below(30)
rsi_30_exits = rsi_30.rsi_crossed_above(70)

entries = [ma_1_entries, ma_2_entries, rsi_14_entries, rsi_30_entries]
exits = [ma_1_exits, ma_2_exits, rsi_14_exits, rsi_30_exits]

starting_weights = generate_weights(weight_length=len(entries))

print(starting_weights)

[0.25 0.25 0.25 0.25]


In [5]:
family = EvolutionaryPortfolioFamily(btc_price, starting_weights, entries, exits, entry_threshold=0.15, exit_threshold=0.15, num_portfolios=5)

In [6]:
log_file = data_file / "results_log.csv"

family.run_simulation(10000, 100, temperature=1, results_log=log_file)

## Results

After running for 10000 steps, below is the plots. It appears with these settings, a Sharpe Ratio of 1.47 can be achieved with a blend of RSI and Moving Averages. Around the 5000 mark, the Sharpe Ratio was achieved.

One can see as the simulation ran, the model rapidly improved on Sharpe Ratio performance once the MA 25/100's weight began to dominate. Further improvements on the model can likely be achieved with more signals in the blend.

In [7]:
simulation_results = pd.read_csv(log_file)

In [11]:
x_column = simulation_results.columns[0]
y_column = simulation_results.columns[1]

# Create the line plot
fig = px.line(simulation_results, x=x_column, y=y_column, title='Sharpe Ratios Evolution')
fig.show()

In [17]:
x_column = simulation_results.columns[0]
y1_column = simulation_results.columns[2]
y2_column = simulation_results.columns[3]
y3_column = simulation_results.columns[4]
y4_column = simulation_results.columns[5]

# Create the figure
fig = go.Figure()

# Add the first line trace
fig.add_trace(go.Scatter(
    x=simulation_results[x_column],
    y=simulation_results[y1_column],
    mode='lines',
    name='MA 25/100'  # Custom legend name for the first line
))

# Add the second line trace
fig.add_trace(go.Scatter(
    x=simulation_results[x_column],
    y=simulation_results[y2_column],
    mode='lines',
    name='MA 25/70'  # Custom legend name for the second line
))

fig.add_trace(go.Scatter(
    x=simulation_results[x_column],
    y=simulation_results[y3_column],
    mode='lines',
    name='RSI 15'  # Custom legend name for the second line
))

fig.add_trace(go.Scatter(
    x=simulation_results[x_column],
    y=simulation_results[y4_column],
    mode='lines',
    name='RSI 30'  # Custom legend name for the second line
))


# Update layout to add titles and adjust the legend
fig.update_layout(
    title='Weight Evolution',
    xaxis_title=x_column,
    yaxis_title='Values',
    legend_title='Signals'
)

fig.show()